In [1]:
# !wget https://developer.nvidia.com/compute/cuda/9.0/Prod/local_installers/cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64-deb
# !dpkg -i cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64-deb
# !apt-key add /var/cuda-repo-9-0-local/7fa2af80.pub
# !apt update -q
# !apt install cuda gcc-6 g++-6 -y -q
# !ln -s /usr/bin/gcc-6 /usr/local/cuda/bin/gcc
# !ln -s /usr/bin/g++-6 /usr/local/cuda/bin/g++

In [2]:
# !curl -sSL "https://julialang-s3.julialang.org/bin/linux/x64/1.7/julia-1.7.3-linux-x86_64.tar.gz" -o julia.tar.gz
# !tar -xzf julia.tar.gz -C /usr --strip-components 1
# !rm -rf julia.tar.gz*
# !julia -e 'using Pkg; pkg"add IJulia; precompile"'

# Analyzing RCT reemployment experiment

## Jonathan Roth's DGP

Here we set up a DGP with heterogenous effects. In this example, with is due to Jonathan Roth, we have

$$
E [Y(0) | Z] = - Z, \quad E [Y(1) |Z] = Z, \quad Z \sim N(0,1).
$$

The CATE is

$$
E [Y(1) - Y(0) | Z ]= 2 Z.
$$

and the ATE is

$$
2 E Z = 0.
$$

We would like to estimate ATE as precisely as possible.

An economic motivation for this example could be provided as follows: Let D be the treatment of going to college, and $Z$ academic skills.  Suppose that academic skills cause lower earnings Y(0) in jobs that don't require college degree, and cause higher earnings  Y(1) in jobs that require college degrees. This type of scenario is reflected in the DGP set-up above.



In [3]:
# import Pkg; Pkg.add("Distributions")
# import Pkg; Pkg.add("Tables")
# import Pkg; Pkg.add("TableOperations")
# import Pkg; Pkg.add("StatsBase")
# import Pkg; Pkg.add("FreqTables")
# import Pkg; Pkg.add("Plots")

In [4]:
# Import relevant packages for splitting data
using LinearAlgebra, GLM, DataFrames, Statistics, Random, Distributions, Tables, TableOperations, StatsBase, FreqTables, DataFrames

In [5]:
# Set Seed
# to make the results replicable (generating random numbers)
Random.seed!(12345676)     # set MC seed

n = 1000                # sample size
Z = randn(n, 1)         # generate Z
Y0 = -Z + randn(n, 1)   # conditional average baseline response is -Z
Y1 = Z + randn(n, 1)    # conditional average treatment effect is +Z
D = Int.(rand(Uniform(), n, 1) .< 0.2)   # treatment indicator; only 23% get treated
length(D[D .== 1])*100/length(D[D .== 0])  # treatment indicator; only 23% get treated
mean(D)

0.193

In [6]:
Y = (Y1.*D) + (Y0.*(ones(n,1)-D))    # observed Y
D = D - fill(mean(D),n,1)            # demean D
Z = Z - fill(mean(Z),n,1)            # demean Z

1000×1 Matrix{Float64}:
 -0.28363285703372
  0.9445462163297016
  1.4041340606063741
 -0.31938856407892297
  0.025831391754146213
  0.1959972014079981
  0.8816207236671101
 -0.39714743413485315
  1.8376685506844046
  0.7433093590338038
  ⋮
  1.0523210386262114
  2.0794634308388917
 -0.5830287094568443
  1.2658914618251444
 -0.6210227063793656
  0.5346326339491493
 -1.0336059839504161
  1.3334449811321292
 -2.4689922728893436
 -1.1552826022174127

## Analyze the RCT data with Precision Adjustment

Consider 

*  classical 2-sample approach, no adjustment (CL)
*  classical linear regression adjustment (CRA)
*  interactive regression adjusment (IRA)

Carry out inference using robust inference, using the sandwich formulas (Eicker-Huber-White).  

Observe that CRA delivers estimates that are less efficient than CL (pointed out by Freedman), whereas IRA delivers more efficient approach (pointed out by Lin). In order for CRA to be more efficient than CL, we need the CRA to be a correct model of the conditional expectation function of Y given D and X, which is not the case here.

In [7]:
Z_times_D = Z.*D
X = hcat(D, Z, Z_times_D)
data = DataFrame(X, [:Z, :D, :Z_times_D])

,Z,D,Z_times_D
,Float64,Float64,Float64
1,-0.193,-0.283633,0.0547411
2,-0.193,0.944546,-0.182297
3,-0.193,1.40413,-0.270998
4,-0.193,-0.319389,0.061642
5,0.807,0.0258314,0.0208459
6,0.807,0.195997,0.15817
7,-0.193,0.881621,-0.170153
8,-0.193,-0.397147,0.0766495
9,-0.193,1.83767,-0.35467


In [8]:
# Import packages for OLS regression
using GLM, Plots

In [9]:
data_aux = [Y D Z Z_times_D]
data_aux = DataFrame(data_aux, [:Y, :D, :Z, :Z_times_D])

,Y,D,Z,Z_times_D
,Float64,Float64,Float64,Float64
1,1.11282,-0.193,-0.283633,0.0547411
2,-0.175068,-0.193,0.944546,-0.182297
3,-1.80793,-0.193,1.40413,-0.270998
4,1.19269,-0.193,-0.319389,0.061642
5,0.311297,0.807,0.0258314,0.0208459
6,0.560205,0.807,0.195997,0.15817
7,-2.48048,-0.193,0.881621,-0.170153
8,0.947113,-0.193,-0.397147,0.0766495
9,-1.03423,-0.193,1.83767,-0.35467


In [10]:
fm_1 = @formula(Y ~ D)
fm_2 = @formula(Y ~ D + Z)
fm_3 = @formula(Y ~ D + Z + Z_times_D)

FormulaTerm
Response:
  Y(unknown)
Predictors:
  D(unknown)
  Z(unknown)
  Z_times_D(unknown)

In [11]:
CL_model = lm(fm_1, data_aux)
CRA_model = lm(fm_2, data_aux)  #classical
IRA_model = lm(fm_3, data_aux)  #interactive approach
# Standard deviations for estimators
CL = sqrt(sum((Y - predict(CL_model)).*(Y - predict(CL_model)))./length(Y))
CRA = sqrt(sum((Y - predict(CRA_model)).*(Y - predict(CRA_model)))./length(Y))
IRA = sqrt(sum((Y - predict(IRA_model)).*(Y - predict(IRA_model)))./length(Y))
@show CL
@show CRA
@show IRA

# Check t values of regressors 
@show coeftable(CL_model).cols[4]
@show coeftable(CRA_model).cols[4]
@show coeftable(IRA_model).cols[4]

CL = 1.3933058979667916
CRA = 1.282349167401232
IRA = 1.0120998697288264


(coeftable(CL_model)).cols[4] = [0.0998031131940856, 0.5557096926021292]


(coeftable(CRA_model)).cols[4] = [0.07392148519479515, 0.8080925829531757, 7.512800066286311e-38]
(coeftable(IRA_model)).cols[4] = [0.11724102121374018, 0.177266312390378, 7.279259820512709e-65, 1.745728665485829e-104]


4-element Vector{Float64}:
 0.11724102121374018
 0.177266312390378
 7.279259820512709e-65
 1.745728665485829e-104

In [12]:
# Comparing models
ftest(CL_model.model, CRA_model.model, IRA_model.model)

F-test: 3 models fitted on 1000 observations
──────────────────────────────────────────────────────────────────────
     DOF  ΔDOF        SSR       ΔSSR      R²     ΔR²        F*   p(>F)
──────────────────────────────────────────────────────────────────────
[1]    3        1941.3013             0.0003                          
[2]    4     1  1644.4194  -296.8819  0.1532  0.1529  179.9974  <1e-37
[3]    5     1  1024.3461  -620.0732  0.4725  0.3193  602.9143  <1e-99
──────────────────────────────────────────────────────────────────────

## Using classical standard errors (non-robust) is misleading here.

We don't teach non-robust standard errors in econometrics courses, but the default statistical inference for lm() procedure in R, summary.lm(), still uses 100-year old concepts, perhaps in part due to historical legacy.  

Here the non-robust standard errors suggest that there is not much difference between the different approaches, contrary to the conclusions reached using the robust standard errors.

In [13]:
@show CL_model
@show CRA_model
@show IRA_model

CL_model = StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, CholeskyPivoted{Float64, Matrix{Float64}}}}, Matrix{Float64}}

Y ~ 1 + D

Coefficients:
──────────────────────────────────────────────────────────────────────────
                  Coef.  Std. Error      t  Pr(>|t|)  Lower 95%  Upper 95%
──────────────────────────────────────────────────────────────────────────
(Intercept)  -0.0726548   0.0441043  -1.65    0.0998  -0.159203  0.0138931
D             0.065871    0.111755    0.59    0.5557  -0.15343   0.285172
──────────────────────────────────────────────────────────────────────────
CRA_model = StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, CholeskyPivoted{Float64, Matrix{Float64}}}}, Matrix{Float64}}

Y ~ 1 + D + Z

Coefficients:
─────────────────────────────────────────────────────────────────────────────
                  Coef.  Std. Error       t  Pr(>|t|)  Lower 95%    

IRA_model = StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, CholeskyPivoted{Float64, Matrix{Float64}}}}, Matrix{Float64}}

Y ~ 1 + D + Z + Z_times_D

Coefficients:
────────────────────────────────────────────────────────────────────────────
                  Coef.  Std. Error       t  Pr(>|t|)  Lower 95%   Upper 95%
────────────────────────────────────────────────────────────────────────────
(Intercept)  -0.0502996   0.0320825   -1.57    0.1172  -0.113257   0.0126575
D             0.109863    0.0813693    1.35    0.1773  -0.049812   0.269538
Z            -0.594236    0.0324276  -18.33    <1e-64  -0.65787   -0.530602
Z_times_D     1.93245     0.0787009   24.55    <1e-99   1.77801    2.08689
────────────────────────────────────────────────────────────────────────────


StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, CholeskyPivoted{Float64, Matrix{Float64}}}}, Matrix{Float64}}

Y ~ 1 + D + Z + Z_times_D

Coefficients:
────────────────────────────────────────────────────────────────────────────
                  Coef.  Std. Error       t  Pr(>|t|)  Lower 95%   Upper 95%
────────────────────────────────────────────────────────────────────────────
(Intercept)  -0.0502996   0.0320825   -1.57    0.1172  -0.113257   0.0126575
D             0.109863    0.0813693    1.35    0.1773  -0.049812   0.269538
Z            -0.594236    0.0324276  -18.33    <1e-64  -0.65787   -0.530602
Z_times_D     1.93245     0.0787009   24.55    <1e-99   1.77801    2.08689
────────────────────────────────────────────────────────────────────────────

## Verify Asymptotic Approximations Hold in Finite-Sample Simulation Experiment

In [14]:
Random.seed!(12345676)     # set MC seed
n = 1000
B = 1000

# format of data = float32
CLs = fill(0., B)
CRAs = fill(0., B)
IRAs = fill(0., B)


# formulas for regressions
fm_1 = @formula(Y ~ D)
fm_2 = @formula(Y ~ D + Z)
fm_3 = @formula(Y ~ D + Z + Z_times_D)

# models
CL_model = lm(fm_1, data_aux)
CRA_model = lm(fm_2, data_aux)  #classical
IRA_model = lm(fm_3, data_aux)  #interactive approach


# simulation
for i in 1:B
    Z = randn(n, 1)         # generate Z
    Y0 = -Z + randn(n, 1)   # conditional average baseline response is -Z
    Y1 = Z + randn(n, 1)    # conditional average treatment effect is +Z
    D = Int.(rand(Uniform(), n, 1) .< 0.2)   # treatment indicator; only 23% get treated

    Y = (Y1.*D) + (Y0.*(ones(n,1)-D))    # observed Y

    D = D - fill(mean(D),n,1)            # demean D
    Z = Z - fill(mean(Z),n,1)            # demean Z

    Z_times_D = Z.*D
    X = hcat(D, Z, Z_times_D)
    data_aux = [Y D Z Z_times_D]
    data_aux = DataFrame(data_aux, [:Y, :D, :Z, :Z_times_D])
    
    CLs[i,] = predict(CL_model)[i]
    CRAs[i,] = predict(CRA_model)[i]
    IRAs[i,] = predict(IRA_model)[i]

end


# check  standard deviations
println("Standard deviations for estimators")  
println("CL model: " , sqrt(sum((Y - predict(CL_model)).*(Y - predict(CL_model)))./length(Y)))
println("CRA model: " , sqrt(sum((Y - predict(CL_model)).*(Y - predict(CRA_model)))./length(Y)))
println("IRA model: " , sqrt(sum((Y - predict(CL_model)).*(Y - predict(IRA_model)))./length(Y)))

Standard deviations for estimators
CL model: 

1.3691676947738234
CRA model: 1.3682762995772266
IRA model: 1.3553039955737827
